In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import requests
from datetime import datetime


In [1]:
# Initialize Dash app
app = dash.Dash(__name__)

# Create layout
app.layout = html.Div([
    # Graph component
    dcc.Graph(id='live-graph'),

    # Triggers periodically
    dcc.Interval(
        id='interval-component',
        interval=10_000,
        n_intervals=0
    )
])

# Initialize empty dataframe
df = pd.DataFrame(columns=['timestamp', 'value'])


# Define callback to update graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):  # increment from intervals

    # Request API
    box_id = "6645db6eeb5aad0007a6e4b6"
    url = f"https://api.opensensemap.org/boxes/{box_id}?format=json"
    data = requests.get(url).json()
    value = data['sensors'][1]['lastMeasurement']['value']
    timestamp = datetime.now()

    global df
    df = pd.concat([df, pd.DataFrame([{'timestamp': timestamp, 'value': float(value)}])], ignore_index=True)

    # Plot data
    trace = go.Scatter(x=df['timestamp'], y=df['value'], mode='lines+markers')
    layout = go.Layout(title='Real-time Sensor Data', xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))
    fig = go.Figure(data=[trace], layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


NameError: name 'dash' is not defined

In [ ]:
# Base URL for the OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
# SenseBox ID
senseBoxId = "6645db6eeb5aad0007a6e4b6"
# List of Sensor IDs
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",  # Replace with actual sensor IDs
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameters
params = {
    'format': 'json',            # or 'csv' if you prefer CSV format
    'from-date': '2024-05-18T09:00:00Z',
    #'to-date': datetime.now(),
    'download': 'true',          # This forces a download if format is 'csv'
    'outliers': 'mark',          # Optional: 'mark' or 'replace' if outlier detection is needed
    'outlier-window': 15,        # Optional: default is 15, can be 1-50
    'delimiter': 'comma'         # Optional: 'comma' or 'semicolon' for CSV
}

# Initialize an empty list to hold the data
all_data = []

# Loop through each sensor ID and retrieve data
for sensorId in sensorIds:
    # Construct the endpoint URL
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    
    # Send the GET request to the API
    response = requests.get(endpoint, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert response data to JSON
        data = response.json()
        # Add sensor ID to each measurement
        for measurement in data:
            measurement['Id'] = sensorId
            # Check for outliers and add an 'outlier' column
            if 'outlier' in measurement:
                measurement['outlier'] = measurement['outlier']
            else:
                measurement['outlier'] = None  # If no outlier info, set as None
        # Append data to the list
        all_data.extend(data)
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')

# Convert the list of data to a DataFrame
sense_df = pd.DataFrame(all_data)
sense_df

In [4]:
import psycopg2

# Verbindung zur Datenbank herstellen
conn = psycopg2.connect(
    dbname="sense_data",
    user="postgres",
    password="your_password",
    host="localhost"
)

# Cursor erstellen
cur = conn.cursor()

# SQL-Abfrage definieren
sql_query = "SELECT * FROM sensor_data WHERE sensor_id = %s;"

# Parameter für die Abfrage
sensor_id = '6645db6eeb5aad0007a6e4b7'

# Abfrage ausführen
cur.execute(sql_query, (sensor_id,))

# Ergebnisse abrufen
results = cur.fetchall()

# Ergebnisse anzeigen
for row in results:
    print(row)

# Cursor und Verbindung schließen
cur.close()
conn.close()



(1, '6645db6eeb5aad0007a6e4b7', datetime.datetime(2024, 5, 21, 13, 29, 12, 598000), 17.48, None, 'Temperatur', '°C', '0101000020E6100000DD24068195831E4034A2B437F8FA4940')
(2, '6645db6eeb5aad0007a6e4b7', datetime.datetime(2024, 5, 21, 13, 28, 12, 510000), 17.55, None, 'Temperatur', '°C', '0101000020E6100000DD24068195831E4034A2B437F8FA4940')
(3, '6645db6eeb5aad0007a6e4b7', datetime.datetime(2024, 5, 21, 13, 27, 12, 371000), 17.7, None, 'Temperatur', '°C', '0101000020E6100000DD24068195831E4034A2B437F8FA4940')
(4, '6645db6eeb5aad0007a6e4b7', datetime.datetime(2024, 5, 21, 13, 26, 12, 211000), 17.62, None, 'Temperatur', '°C', '0101000020E6100000DD24068195831E4034A2B437F8FA4940')
(5, '6645db6eeb5aad0007a6e4b7', datetime.datetime(2024, 5, 21, 13, 25, 12, 182000), 17.77, None, 'Temperatur', '°C', '0101000020E6100000DD24068195831E4034A2B437F8FA4940')
(6, '6645db6eeb5aad0007a6e4b7', datetime.datetime(2024, 5, 21, 13, 24, 12, 147000), 17.77, None, 'Temperatur', '°C', '0101000020E6100000DD24068195

In [ ]:
import dash
from dash import html

app = dash.Dash(__name__)

# Layout definieren
app.layout = html.Div([
    # Streifen auf der linken Seite
    html.Div(
        id='side-panel',
        style={
            'width': '25%',  # Breite des Streifens einstellen
            'position': 'fixed',  # Position fixieren
            'top': 0,  # Oben anordnen
            'left': 0,  # Links anordnen
            'bottom': 0,  # Unten anordnen
            'backgroundColor': '#f0f0f0',  # Hintergrundfarbe einstellen
            'overflowY': 'scroll'  # Scrollen ermöglichen, wenn der Inhalt überläuft
        },
        children=[
            # Inhalte des Streifens
            html.Div([
                html.H2('Umweltmonitoring SS24 Projektarbeit'),
                html.P('Hier kannst du verschiedene Informationen anzeigen lassen.'),
                # Beispielbild mit angepasster Größe
                html.Img(
                    src='https://docs.sensebox.de/images/2020-10-16-opensensemap-faq/openSenseMap_github.png',
                    style={
                        'width': '90px',  # Breite des Bildes einstellen
                        'height': '40px'  # Höhe des Bildes einstellen
                    }
                )
            ], style={'padding': '20px'})
        ]
    ),
    
    # Hauptinhalt auf der rechten Seite
    html.Div([
        html.H1('Hauptinhalt'),
        html.P('Hier befindet sich der Hauptinhalt der Seite.')
    ], style={'marginLeft': '40%', 'padding': '10px'})  # Linken Rand einstellen, um Platz für den Streifen zu machen
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [11]:
import requests

def get_current_temperature(box_id):
    url = f"https://api.opensensemap.org/boxes/{box_id}"
    try:
        response = requests.get(url)
        data = response.json()
        temperature = data['sensors'][0]['lastMeasurement']['value']
        return temperature
    except Exception as e:
        print("Fehler beim Abrufen der Temperatur:", e)
        return None

# Hier die ID deiner OpenSenseMap-Box eintragen
box_id = "6645db6eeb5aad0007a6e4b6"

temperature = get_current_temperature(box_id)
if temperature is not None:
    print("Die aktuelle Temperatur beträgt:", temperature, "°C")


Die aktuelle Temperatur beträgt: 17.36 °C


In [17]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import requests

# Basis-URL für die OpenSenseMap API
BASE_URL = "https://api.opensensemap.org/boxes"
SENSE_BOX_ID = "6645db6eeb5aad0007a6e4b6"
SENSOR_IDS = [
    "6645db6eeb5aad0007a6e4b7",
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameter für die API-Anfrage
PARAMS = {
    'format': 'json',
    'from-date': '2024-05-18T09:00:00Z',
    'download': 'false',
    'outliers': 'mark',
    'outlier-window': 15
}

SENSOR_INFO = {
    '6645db6eeb5aad0007a6e4b7': {'name': 'Temperatur', 'unit': '°C'},
    '6645db6eeb5aad0007a6e4b8': {'name': 'rel. Luftfeuchte', 'unit': '%'},
    '6645db6eeb5aad0007a6e4b9': {'name': 'Luftdruck', 'unit': 'hPa'},
    '6645db6eeb5aad0007a6e4ba': {'name': 'UV-Intensität', 'unit': 'μW/cm²'},
}

def fetch_sensor_data(sensor_id):
    endpoint = f'{BASE_URL}/{SENSE_BOX_ID}/data/{sensor_id}'
    response = requests.get(endpoint, params=PARAMS)
    if response.status_code == 200:
        data = response.json()
        for measurement in data:
            measurement['Id'] = sensor_id
            measurement['outlier'] = measurement.get('outlier', None)
        return data
    else:
        raise Exception(f'Error retrieving data for sensor {sensor_id}: {response.status_code}')

def fetch_all_sensor_data(sensor_ids):
    all_data = []
    for sensor_id in sensor_ids:
        all_data.extend(fetch_sensor_data(sensor_id))
    return all_data

def preprocess_data(data):
    df = pd.DataFrame(data)
    df['createdAt'] = pd.to_datetime(df['createdAt'])
    df['sensor_name'] = df['Id'].map(lambda x: SENSOR_INFO[x]['name'])
    df['unit'] = df['Id'].map(lambda x: SENSOR_INFO[x]['unit'])
    df = df[df['outlier'] != True]
    return df

def pivot_data(df):
    return df.pivot_table(index='createdAt', columns='sensor_name', values='value').reset_index()

def create_figure(df, x, y, title, y_title, y2=None, y2_title=None):
    fig = px.line(df, x=x, y=y, title=title)
    fig.update_xaxes(title_text='Messzeitpunkte', tickangle=45, tickformat='%Y-%m-%d')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(title_text=y_title)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")])
    if y2 and y2_title:
        fig.add_scatter(x=df[x], y=df[y2], mode='lines', name=y2_title, yaxis='y2')
        fig.update_layout(yaxis2=dict(title=y2_title, overlaying='y', side='right'))
    return fig

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Div(
        id='side-panel',
        style={
            'width': '25%',  
            'position': 'fixed',  
            'top': 0,  
            'left': 0,  
            'bottom': 0,  
            'backgroundColor': '#f0f0f0', 
            'overflowY': 'scroll'  
        },
        children=[
            html.Div([
                html.H2('Umweltmonitoring SS24 Projektarbeit'),
                html.P('Im Rahmen eines Fachpraktikums wurde eine OpenSenseBox zur Umweltüberwachung aufgebaut, um Daten wie Luftqualität und Temperatur zu erfassen und zu visualisieren. Die Echtzeitvisualisierung bietet Einblicke in lokale Umweltbedingungen. Ein weiteres Ziel ist die Entwicklung eines Modells zur Wettervorhersage basierend auf diesen Daten.'),
                html.Img(
                    src='https://docs.sensebox.de/images/2020-10-16-opensensemap-faq/openSenseMap_github.png',
                    style={
                        'width': '200px',  
                        'height': '90px'
                    }
                ),
                html.P("Sensoren:"),
                html.Ul([
                    html.Li(f"{SENSOR_INFO[sensor_id]['name']} ({SENSOR_INFO[sensor_id]['unit']})")
                    for sensor_id in SENSOR_IDS
                ]),
                html.P("Beispiel text"),
            ], style={'padding': '20px'}),
            html.Footer("Gruppe: Alexandru, Evelyn, Rafael", style={'textAlign': 'left', 'paddingLeft': '20px'})  
        ]
    ),
    
    # Main content on the right side
    html.Div([
        dbc.Container([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=temp_figure), width=6),
                dbc.Col(dcc.Graph(figure=temp_humid_figure), width=6)
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=temp_uv_figure), width=6),
                dbc.Col(dcc.Graph(figure=pressure_humid_figure), width=6)
            ])
        ], fluid=True)
    ], style={'marginLeft': '25%', 'padding': '10px'})  # Adjust left margin to make space for the sidebar
])

if __name__ == '__main__':
    app.run_server(debug=True)


NameError: name 'temp_figure' is not defined

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import requests

# Base URL for the OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
# SenseBox ID
senseBoxId = "6645db6eeb5aad0007a6e4b6"
# List of Sensor IDs
sensorIds = [
    "6645db6eeb5aad0007a6e4b7",
    "6645db6eeb5aad0007a6e4b8",
    "6645db6eeb5aad0007a6e4b9",
    "6645db6eeb5aad0007a6e4ba"
]

# Parameters
params = {
    'format': 'json',
    'from-date': '2024-05-18T09:00:00Z',
    'download': 'false',
    'outliers': 'mark',
    'outlier-window': 15
}

sensor_info = {
    '6645db6eeb5aad0007a6e4b7': {'name': 'Temperatur', 'unit': '°C'},
    '6645db6eeb5aad0007a6e4b8': {'name': 'rel. Luftfeuchte', 'unit': '%'},
    '6645db6eeb5aad0007a6e4b9': {'name': 'Luftdruck', 'unit': 'hPa'},
    '6645db6eeb5aad0007a6e4ba': {'name': 'UV-Intensität', 'unit': 'μW/cm²'},
}

# Function to fetch data from OpenSenseMap API
def fetch_sensor_data(sensorId):
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        data = response.json()
        for measurement in data:
            measurement['Id'] = sensorId
            measurement['outlier'] = measurement.get('outlier', None)
        return data
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')
        return []

# Initialize an empty list to hold the data
all_data = []

# Loop through each sensor ID and retrieve data
for sensorId in sensorIds:
    all_data.extend(fetch_sensor_data(sensorId))

# Convert the list of data to a DataFrame
sense_df = pd.DataFrame(all_data)
sense_df['createdAt'] = pd.to_datetime(sense_df['createdAt'])
sense_df['sensor_name'] = sense_df['Id'].map(lambda x: sensor_info[x]['name'])
sense_df['unit'] = sense_df['Id'].map(lambda x: sensor_info[x]['unit'])

# Filter out outliers
sense_df = sense_df[sense_df['outlier'] != True]

# Pivot the DataFrame for easier plotting
opensense_df = sense_df.pivot_table(
    index='createdAt', 
    columns='sensor_name', 
    values='value'
).reset_index()

# Anzahl der Messpunkte
num_measurements = len(sense_df)
# Anzahl der Fehlerhaften Messungen
num_outliers = sense_df['outlier'].sum()
# Prozentsatz der Outliers
percent_outliers = round((num_outliers / num_measurements) * 100, 2)

# Function to create a figure
def create_figure(df, x, y, title, y_title, y2_title=None):
    fig = px.line(df, x=x, y=y, title=title)
    fig.update_xaxes(title_text='Messzeitpunkte')
    fig.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')
    fig.update_layout(
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey'),
        annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.0, showarrow=False, text="Hinweis: Ausreißer wurden entfernt")]
    )
    if y2_title:
        fig.add_scatter(x=df[x], y=df[y2_title], mode='lines', name=y2_title, yaxis='y2')
        fig.update_layout(yaxis=dict(title=y_title), yaxis2=dict(title=y2_title, overlaying='y', side='right'))
    else:
        fig.update_yaxes(title_text=y_title)
    return fig

# Create the figures
temp_figure = create_figure(opensense_df, 'createdAt', 'Temperatur', 'Temperaturverlauf (°C) über Zeit', 'Temperatur (°C)')
temp_humid_figure = create_figure(opensense_df, 'createdAt', ['Temperatur'], 'Temperatur und rel. Luftfeuchte über Zeit', 'Temperatur (°C)', 'rel. Luftfeuchte')
temp_uv_figure = create_figure(opensense_df, 'createdAt', ['Temperatur'], 'Temperatur und UV-Intensität über Zeit', 'Temperatur (°C)', 'UV-Intensität')
pressure_humid_figure = create_figure(opensense_df, 'createdAt', ['Luftdruck'], 'Luftdruck und rel. Luftfeuchte über Zeit', 'Luftdruck (hPa)', 'rel. Luftfeuchte')

# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout definition
app.layout = html.Div([
    # Sidebar on the left side
    html.Div(
        id='side-panel',
        style={
            'width': '25%',  
            'position': 'fixed',  
            'top': 0,  
            'left': 0,  
            'bottom': 0,  
            'backgroundColor': '#f0f0f0', 
            'overflowY': 'scroll'  
        },
        children=[
            html.Div([
                html.H2('Umweltmonitoring SS24 Projektarbeit'),
                html.P('Im Rahmen eines Fachpraktikums wurde eine OpenSenseBox zur Umweltüberwachung aufgebaut, um Daten wie Luftqualität und Temperatur zu erfassen und zu visualisieren. Die Echtzeitvisualisierung bietet Einblicke in lokale Umweltbedingungen. Ein weiteres Ziel ist die Entwicklung eines Modells zur Wettervorhersage basierend auf diesen Daten.'),
                html.Img(
                    src='https://docs.sensebox.de/images/2020-10-16-opensensemap-faq/openSenseMap_github.png',
                    style={
                        'width': '200px',  
                        'height': '90px'
                    }
                ),
                html.P(f"Anzahl der Messpunkte: {num_measurements}"),
                html.P(f"Anzahl der Fehlerhaften Messungen: {num_outliers}"),
                html.P(f"Anteil der Fehlerhaften Messungen: {percent_outliers}%"),
                html.P("Sensoren:"),
                html.Ul([
                    html.Li(f"{sensor_info[sensor_id]['name']} ({sensor_info[sensor_id]['unit']})")
                    for sensor_id in sensorIds
                ]),
                html.P("Beispiel text"),
            ], style={'padding': '20px'}),
            html.Footer("Gruppe: Alexandru, Evelyn, Rafael", style={'textAlign': 'left', 'paddingLeft': '20px'})  
        ]
    ),
    
    # Main content on the right side
    html.Div([
        dbc.Container([
            dbc.Row([
                dbc.Col(dcc.Graph(id='temp-figure', figure=temp_figure), width=6),
                dbc.Col(dcc.Graph(id='temp-humid-figure', figure=temp_humid_figure), width=6)
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='temp-uv-figure', figure=temp_uv_figure), width=6),
                dbc.Col(dcc.Graph(id='pressure-humid-figure', figure=pressure_humid_figure), width=6)
            ])
        ], fluid=True)
    ], style={'marginLeft': '25%', 'padding': '10px'})  
])

# Interval component for auto-updating the graphs
interval = dcc.Interval(id='interval-component', interval=60000, n_intervals=0)

# Add interval component to the layout
app.layout.children.append(interval)

# Callback function to update the figures
@app.callback(
    [Output('temp-figure', 'figure'),
     Output('temp-humid-figure', 'figure'),
     Output('temp-uv-figure', 'figure'),
     Output('pressure-humid-figure', 'figure')],
    [Input('interval-component', 'n_intervals')]
)
def update_figures(n_intervals):
    all_data = []
    for sensorId in sensorIds:
        all_data.extend(fetch_sensor_data(sensorId))

    sense_df = pd.DataFrame(all_data)
    sense_df['createdAt'] = pd.to_datetime(sense_df['createdAt'])
    sense_df['sensor_name'] = sense_df['Id'].map(lambda x: sensor_info[x]['name'])
    sense_df['unit'] = sense_df['Id'].map(lambda x: sensor_info[x]['unit'])

    sense_df = sense_df[sense_df['outlier'] != True]

    opensense_df = sense_df.pivot_table(
        index='createdAt',
        columns='sensor_name',
        values='value'
    ).reset_index()

    temp_figure = create_figure(opensense_df, 'createdAt', 'Temperatur', 'Temperaturverlauf (°C) über Zeit', 'Temperatur (°C)')
    temp_humid_figure = create_figure(opensense_df, 'createdAt', ['Temperatur'], 'Temperatur und rel. Luftfeuchte über Zeit', 'Temperatur (°C)', 'rel. Luftfeuchte')
    temp_uv_figure = create_figure(opensense_df, 'createdAt', ['Temperatur'], 'Temperatur und UV-Intensität über Zeit', 'Temperatur (°C)', 'UV-Intensität')
    pressure_humid_figure = create_figure(opensense_df, 'createdAt', ['Luftdruck'], 'Luftdruck und rel. Luftfeuchte über Zeit', 'Luftdruck (hPa)', 'rel. Luftfeuchte')

    return temp_figure, temp_humid_figure, temp_uv_figure, pressure_humid_figure

if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/hl/g54whv_j0vd21tv5fjmrvw6m0000gn/T/ipykernel_71065/1658779958.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/hl/g54whv_j0vd21tv5fjmrvw6m0000gn/T/ipykernel_71065/1658779958.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'